In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import torch.nn as nn
from tqdm import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Importar el csv de entrenamiento a una variable de pandas.
df_train = pd.read_csv('/content/train.csv')
df_test = pd.read_csv('/content/test.csv')

In [ ]:
Y_train = df_train['label'].values
X_train = df_train.drop('label', axis=1).values

# X es el conjunto de datos, e Y es la etiqueta
X_train, X_test, Y_train, Y_test = train_test_split(X_train, Y_train, test_size=0.2)

In [ ]:

X_train = torch.from_numpy(X_train).float()
X_test = torch.from_numpy(X_test).float()
Y_train = torch.from_numpy(Y_train).long()
Y_test = torch.from_numpy(Y_test).long()

X_train = X_train.view(-1, 1, 28, 28) #linea anterior: X_train = X_train.view(-1, 28, 28)
X_test = X_test.view(-1, 1, 28, 28) #linea anterior: X_test = X_test.view((-1, 28, 28)


In [ ]:
dataloader = {
    'train': torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset( 
            X_train,
            Y_train
        ),
        batch_size=64,
        shuffle=True
    ),
    'test': torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(
          X_test,
          Y_test
        ),
        batch_size=64,
        shuffle=True
    )
}

## Model 1

In [ ]:
def block(c_in, c_out, k=3, p=1, s=1, pk=2, ps=2):
    return torch.nn.Sequential(
        torch.nn.Conv2d(c_in, c_out, k, padding=p, stride=s),
        torch.nn.ReLU(),
        torch.nn.MaxPool2d(pk, stride=ps)
    )

def block2(c_in, c_out):
    return torch.nn.Sequential(
        torch.nn.Linear(c_in, c_out),
        torch.nn.ReLU()
    )

class CNN(torch.nn.Module):
  def __init__(self, n_channels=1, n_outputs=10):
    super().__init__()
    self.conv1 = block(n_channels, 64)
    self.conv2 = block(64, 128)
    self.fc = torch.nn.Linear(128*7*7, n_outputs)

  def forward(self, x):
    x = self.conv1(x)
    x = self.conv2(x)
    x = x.view(x.shape[0], -1)
    x = self.fc(x)
    return x

In [ ]:
def fit(model, dataloader, epochs=5):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    criterion = torch.nn.CrossEntropyLoss()
    for epoch in range(1, epochs+1):
        model.train()
        train_loss, train_acc = [], []
        bar = tqdm(dataloader['train'])
        for batch in bar:
            X, y = batch
            X, y = X.to(device), y.to(device)
            optimizer.zero_grad()
            y_hat = model(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
            train_acc.append(acc)
            bar.set_description(f"loss {np.mean(train_loss):.5f} acc {np.mean(train_acc):.5f}")
        bar = tqdm(dataloader['test'])
        val_loss, val_acc = [], []
        model.eval()
        with torch.no_grad():
            for batch in bar:
                X, y = batch
                X, y = X.to(device), y.to(device)
                y_hat = model(X)
                loss = criterion(y_hat, y)
                val_loss.append(loss.item())
                acc = (y == torch.argmax(y_hat, axis=1)).sum().item() / len(y)
                val_acc.append(acc)
                bar.set_description(f"val_loss {np.mean(val_loss):.5f} val_acc {np.mean(val_acc):.5f}")
        print()
        print(f"Epoch {epoch}/{epochs} loss {np.mean(train_loss):.5f} val_loss {np.mean(val_loss):.5f} acc {np.mean(train_acc):.5f} val_acc {np.mean(val_acc):.5f}")

In [ ]:
model = CNN()

In [ ]:
fit(model, dataloader)

val_loss 0.16090 val_acc 0.95407: 100%|██████████| 132/132 [00:09<00:00, 13.67it/s]



Epoch 1/5 loss 0.63836 val_loss 0.16090 acc 0.93182 val_acc 0.95407


val_loss 0.14344 val_acc 0.96697: 100%|██████████| 132/132 [00:10<00:00, 12.58it/s]



Epoch 2/5 loss 0.11116 val_loss 0.14344 acc 0.96854 val_acc 0.96697


val_loss 0.10551 val_acc 0.97242: 100%|██████████| 132/132 [00:09<00:00, 13.29it/s]



Epoch 3/5 loss 0.10301 val_loss 0.10551 acc 0.97119 val_acc 0.97242


val_loss 0.11099 val_acc 0.97408: 100%|██████████| 132/132 [00:09<00:00, 14.16it/s]



Epoch 4/5 loss 0.08937 val_loss 0.11099 acc 0.97473 val_acc 0.97408


val_loss 0.10848 val_acc 0.97384: 100%|██████████| 132/132 [00:09<00:00, 13.31it/s]


Epoch 5/5 loss 0.06569 val_loss 0.10848 acc 0.98173 val_acc 0.97384


In [ ]:
df_test = torch.from_numpy(df_test.to_numpy()).float()

df_test = df_test.view(-1, 1, 28, 28) 

In [ ]:
dataloader_predict = {
    'predict': torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(df_test),
        batch_size=64
    )
}

In [ ]:
bar = tqdm(dataloader_predict['predict'])
predict = []
for batch in bar:
    for tensor in batch:
      X = tensor
      X = X.to(device)
      y_hat = model(X)
      predict.append(torch.argmax(y_hat, axis=1))
      bar.set_description()

100%|██████████| 438/438 [00:32<00:00, 13.35it/s]


In [ ]:
predicts = torch.cat(predict, dim=0)

In [ ]:
ImageId = list(range(1,28001))

In [ ]:
sample =  pd.DataFrame({"ImageId":ImageId, "Label":predicts})

In [ ]:
sample.to_csv("Predicciones_MNIST.csv", index=False)